In [1]:
!nvidia-smi

Mon Nov  7 12:26:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Clone repository

In [2]:
!git clone https://github.com/kaylode/vehicle-counting.git main
%cd main

Cloning into 'main'...
remote: Enumerating objects: 1033, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 1033 (delta 22), reused 22 (delta 19), pack-reused 992
Receiving objects: 100% (1033/1033), 263.56 MiB | 17.48 MiB/s, done.
Resolving deltas: 100% (512/512), done.
/content/main


In [3]:
%%capture
!pip install -r requirements.txt

In [4]:
!pip uninstall opencv-python-headless
!pip install opencv-python-headless==4.1.2.30

Found existing installation: opencv-python-headless 4.6.0.66
Uninstalling opencv-python-headless-4.6.0.66:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/cv2/*
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless-4.6.0.66.dist-info/*
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libavcodec-5896f664.so.58.134.100
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libavformat-8ef5c7db.so.58.76.100
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libavutil-9c768859.so.56.70.100
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libbz2-a273e504.so.1.0.6
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libcrypto-d21001fc.so.1.1
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libgfortran-91cc3cb1.so.3.0.0
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libopenblas-r0-f650aae0.3.3.so
    /usr/local/lib/python3.7/dis

In [5]:
!git reset --hard HEAD
!git pull

HEAD is now at 5007a54 Merge pull request #17 from kaylode/fix
Already up to date.


# Copy video file
For input data, you need 2 file:
- Video
- Directions and Zone annotation file associated with the street in the video, have same name with the video


# Start inference
Define these things before run:
- Path to street annotation files in **configs/cam_configs.yaml**
- Path to video file
- Model's checkpoint

In [6]:
!gdown https://drive.google.com/uc?id=1-Y6H3QdRevfBKYDQxgRiR2CRinRVPt9O -O /content/yolov5s.pt

Traceback (most recent call last):
  File "/usr/local/bin/gdown", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/gdown/cli.py", line 105, in main
    use_cookies=not args.no_cookies,
  File "/usr/local/lib/python3.7/dist-packages/gdown/download.py", line 114, in download
    res = sess.get(url, headers=headers, stream=True)
  File "/usr/local/lib/python3.7/dist-packages/requests/sessions.py", line 543, in get
    return self.request('GET', url, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/requests/sessions.py", line 516, in request
    prep = self.prepare_request(req)
  File "/usr/local/lib/python3.7/dist-packages/requests/sessions.py", line 459, in prepare_request
    hooks=merge_hooks(request.hooks, self.hooks),
  File "/usr/local/lib/python3.7/dist-packages/requests/models.py", line 314, in prepare
    self.prepare_url(url, params)
  File "/usr/local/lib/python3.7/dist-packages/requests/models.py", line 388, in prepare_url
    rai

In [7]:
!mkdir /content/results

In [8]:
%cd main
!python run.py --input_path='/content/main/demo/sample/cam_04.mp4' \
              --output_path="/content/results" \
              --weight="/content/yolov5s.pt"

[Errno 2] No such file or directory: 'main'
/content/main
##########   CONFIGURATION INFO   ##########
model_name:	yolov5s
gpu_devices:	0
num_epochs:	300
batch_size:	16
num_workers:	2
min_iou:	0.45
min_conf:	0.25
max_det:	300
image_size:	[640, 640]
keep_ratio:	True


Nubmer of gpus: 1
CUDA:0 (Tesla T4, 15109.75MB)

/usr/local/lib/python3.7/dist-packages/torch/hub.py:267: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  "You are about to download and run code from an untru

# Tracking results
There will be two result files:
- A **.csv** contains tracking result in every frames
- A **.mp4** plots the results from .csv to original video

In [9]:
import pandas as pd
pd.read_csv('/content/results/cam_04.csv').head()

,track_id,frame_id,box,color,label,direction,fpoint,lpoint,fframe,lframe
0,2,3,"[856, 382, 883, 441]","(255, 255, 240)",0,1,"(869.5, 411.5)","(955.0, 354.0)",3,31
1,2,4,"[866, 378, 892, 434]","(255, 255, 240)",0,1,"(869.5, 411.5)","(955.0, 354.0)",3,31
2,2,5,"[869, 377, 894, 431]","(255, 255, 240)",0,1,"(869.5, 411.5)","(955.0, 354.0)",3,31
3,2,6,"[869, 376, 895, 431]","(255, 255, 240)",0,1,"(869.5, 411.5)","(955.0, 354.0)",3,31
4,2,7,"[872, 375, 897, 429]","(255, 255, 240)",0,1,"(869.5, 411.5)","(955.0, 354.0)",3,31
